# Calculating the Friction Surface for Rhode Island

In [ ]:
import geopyspark as gps
import numpy as np
import pyproj
import fiona

from functools import partial
from shapely.geometry import shape, MultiPoint, MultiLineString
from shapely.ops import transform
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from geonotebook.wrappers import VectorData, TMSRasterData

conf = gps.geopyspark_conf(appName="gps-osm-ingest", master='yarn')
conf.set('spark.ui.enabled', True)

sc = SparkContext(conf=conf)

hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoopConf.set("fs.s3.awsAccessKeyId", 'your_access_key')
hadoopConf.set("fs.s3.awsSecretAccessKey", 'your_secret_key')

pysc = gps.get_spark_context()
session = SparkSession.builder.config(conf=pysc.getConf()).enableHiveSupport().getOrCreate()

In [ ]:
M.set_center(-71.88333333333334, 41.15, 8)

In [ ]:
rhode_island_extent = gps.Extent(-71.88333333333334, 41.15, -71.11666666666666, 42.016666666666666)

## Calculating the Friction Surface From OSM, NLCD, NHD, and NED Data

### Reading and Formatting the OSM Data

In [ ]:
# Download the orc file from S3

!curl -o /tmp/rhode-island.orc https://s3.amazonaws.com/geotrellis-test/xterrain/rhode-island.orc

In [ ]:
# This cell contains the logic that assigns each section of road a
# speed based on the type of road that section is.

default_speeds = {
'motorway':65,
'trunk':45,
'primary':40,
'secondary':35,
'tertiary':30,
'unclassified':20,
'residential':20,
'service':15,
'motorway_link':45,
'trunk_link':40,
'primary_link':35,
'secondary_link':30,
'tertiary_link':25,
'living_street':5,
'pedestrian':5,
'track':15,
'road':20}

words = ['maxspeed', 'ambiguous', 'signals', 
         'none', 'walk', 'variable', 
         'national', 'fixme', 'unposted', 'implicit']

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def default_speed(highway):
    if not highway in default_speeds:
        return default_speeds['road']
    else:
        return default_speeds[highway]

def get_maxspeed(speed, units, highway):
    speeds = speed.split(';|,-')
    maxspeed = 0
    for sp in speeds:
        sp = sp.replace(units, '')
        if (is_number(sp)):
            if units == 'kph':
                sp = float(sp) / 1.609344 
            elif units == 'knots':
                sp = 0.868976 * float(knots)
            else:
                sp = float(sp)
                
            if sp > maxspeed:
                maxspeed = sp
    if maxspeed > 0:
        speed = maxspeed
    else:
        speed = default_speed(highway)

    return speed

def get_highway_cellvalue(osm_feature):   
    highway = osm_feature.properties.tags['highway']
    speed = osm_feature.properties.tags.get('maxspeed', '')
                                
    speed = speed.lower().strip()
        
    # if we don't have a speed, give it a default
    if len(speed) == 0:
        speed = default_speed(highway)
    elif not is_number(speed):
        if 'kph' in speed:
            speed = get_maxspeed(speed, 'kph', highway)
        elif 'km/h' in speed:
            speed = get_maxspeed(speed, 'km/h', highway)
        elif 'kmh' in speed:
            speed = get_maxspeed(speed, 'kmh', highway)
        elif 'mph' in speed:
            speed = get_maxspeed(speed, 'mph', highway)
        elif 'knots' in speed:
            speed = get_maxspeed(speed, 'knots', highway)
        elif speed in words:
            speed = default_speed(highway)
        else:
            speed = get_maxspeed(speed, '', highway)            
    if float(speed) <= 0.0:
        speed = default_speed(highway)

    speed = float(speed)
    return gps.CellValue(speed, speed)

In [ ]:
file_uri = "s3://geotrellis-test/xterrain/rhode-island.orc"
osm_dataframe = session.read.orc(file_uri)

In [ ]:
# Get all of the lines that are contained within the DataFrame

osm = gps.vector_pipe.osm_reader.from_dataframe(osm_dataframe)
lines = osm.get_line_features_rdd()

In [ ]:
# Only roads/paths are of interest
highways = lines.filter(lambda feature: 'highway' in feature.properties.tags)

In [ ]:
path_tags = ['footway', 'steps', 'bridleway', 'path']

# Filter out the highways into roads and paths

roads = highways.filter(lambda feature: feature.properties.tags['highway'] not in path_tags)
paths = highways.filter(lambda feature: feature.properties.tags['highway'] in path_tags)

In [ ]:
path_features = paths.map(lambda feature: gps.Feature(feature.geometry, gps.CellValue(3.74, 3.74)))

In [ ]:
# Encode the road speeds as feature properties for rasterization
road_features = roads.map(lambda feature: gps.Feature(feature.geometry, get_highway_cellvalue(feature)))

In [ ]:
road_raster = gps.geotrellis.rasterize_features(
    features = road_features,
    crs = 4326,
    zoom = 10,
    cell_type=gps.CellType.INT8RAW,
    partition_strategy = gps.SpatialPartitionStrategy(32)
).convert_data_type(gps.CellType.FLOAT32, -2147483648.0)

In [ ]:
path_raster = gps.geotrellis.rasterize_features(
    features = path_features,
    crs = 4326,
    zoom = 10,
    cell_type=gps.CellType.INT8RAW,
    partition_strategy = gps.SpatialPartitionStrategy(32)
).convert_data_type(gps.CellType.FLOAT32, -2147483648.0)

In [ ]:
tiled_path_raster = path_raster.tile_to_layout(road_raster.layer_metadata)

### Displaying the Rasterized Roads

In [ ]:
color_map = gps.ColorMap.from_colors(
    breaks = np.arange(8, 100, 4), 
    color_list = gps.get_colors_from_matplotlib('magma'))

osm_wm = road_raster.tile_to_layout(gps.GlobalLayout(tile_size=256), target_crs=3857)

layer = gps.TMS.build(osm_wm.pyramid(), color_map)
M.add_layer(TMSRasterData(layer), name="OSM-roads")

In [ ]:
M.remove_layer(M.layers[0])

### Reading and Formatting the NLCD Data

In [ ]:
nlcd_pmts_map = {
    11.0: 0.0,
    12.0: 0.15,
    21.0: 0.9,
    22.0: 0.9,
    23.0: 0.9,
    24.0: 0.95,
    31.0: 0.1,
    41.0: 0.7,
    42.0: 0.65,
    43.0: 0.75,
    51.0: 0.75,
    52.0: 0.75,
    71.0: 0.8,
    81.0: 0.8,
    82.0: 0.8,
    90.0: 0.2,
    95.0: 0.25
}

In [ ]:
# Reading NLCD Data
nlcd = gps.geotiff.get(gps.LayerType.SPATIAL, "s3://gt-rasters/nlcd/2011/tiles", crs="epsg:4326", max_tile_size=256, num_partitions=32)

# Tile NLCD layer to same layout as rasterized OSM features
tiled_nlcd = nlcd.tile_to_layout(road_raster.layer_metadata, partition_strategy=gps.SpatialPartitionStrategy(32))

masked_nlcd = tiled_nlcd.mask(rhode_island_extent.to_polygon)

In [ ]:
nlcd_pmts = masked_nlcd.reclassify(value_map=nlcd_pmts_map,
                                   data_type=float,
                                   classification_strategy=gps.ClassificationStrategy.EXACT
                                  ).convert_data_type(gps.CellType.FLOAT32, 0.0)

### Displaying the NLCD Data

In [ ]:
nlcd_tms = nlcd_pmts.tile_to_layout(gps.GlobalLayout(tile_size=256), target_crs=3857)

color_map = gps.ColorMap.build(nlcd_tms.get_histogram(), 'magma')

layer = gps.TMS.build(nlcd_tms.pyramid(), color_map)
M.add_layer(TMSRasterData(layer), name="NLCD-Data")

In [ ]:
M.remove_layer(M.layers[0])

### Reading and Formatting the NHD Data

In [ ]:
nhd_pmts_map = {
    0: 0.0,
    1: 0.7,
    2: 0.6,
    3: 0.5,
    4: 0.3,
    5: 0.2,
    6: 0.1
}

def assign_nhd_cell_values(order):
    pmt = nhd_pmts_map[order]
    return gps.CellValue(pmt, pmt)

In [ ]:
nhd = gps.shapefile.get("s3://geotrellis-test/jpolchlopek/NHDPlus/NHDPlus_H_0109/NHDFlowlines.shp").filter(lambda feat: 'n.FType' in feat.properties and (feat.properties['n.FType'] == '336' or feat.properties['n.FType'] == '460'))

In [ ]:
nhd_filt_features = nhd.flatMap(lambda feat: 
    [gps.Feature(feat.geometry, assign_nhd_cell_values(int(feat.properties['p.StreamOr'])))] if 'p.StreamOr' in feat.properties and feat.properties['p.StreamOr'] and int(feat.properties['p.StreamOr']) >= 0 else []
)

In [ ]:
nhd_raster = gps.geotrellis.rasterize_features(
    features = nhd_filt_features,
    crs = 4326,
    zoom = 10,
    cell_type=gps.CellType.FLOAT32RAW,
    partition_strategy = gps.SpatialPartitionStrategy(32)
)

In [ ]:
tiled_nhd = nhd_raster.tile_to_layout(road_raster.layer_metadata, partition_strategy=gps.SpatialPartitionStrategy(32))

In [ ]:
masked_nhd = tiled_nhd.mask(rhode_island_extent.to_polygon, partition_strategy=gps.SpatialPartitionStrategy(32))

### Displaying the Rasterized NHD Features

In [ ]:
color_map = gps.ColorMap.from_colors(
    breaks = np.arange(0.1, 1.0, 0.1), 
    color_list = gps.get_colors_from_matplotlib('magma'))

tsm_nhd = masked_nhd.tile_to_layout(gps.GlobalLayout(tile_size=256), target_crs=3857)

#layer = gps.TMS.build(tsm_nhd.convert_data_type(gps.CellType.FLOAT32, 0.0).pyramid(), color_map)
layer = gps.TMS.build(tsm_nhd.pyramid(), color_map)

M.add_layer(TMSRasterData(layer), name="NHD-DATA")

In [ ]:
M.remove_layer(M.layers[0])

### Reading and Formatting the NED Data

In [ ]:
ned_location = 's3://azavea-datahub/raw/ned-13arcsec-geotiff/'
ned_files = ['{}imgn{}w0{}_13.tif'.format(ned_location, n, e) for n in [41, 42, 43] for e in [71, 72]]

In [ ]:
ned_raw = gps.geotiff.get(gps.LayerType.SPATIAL, ned_files, num_partitions=32, max_tile_size=256)

In [ ]:
tiled_ned = ned_raw.tile_to_layout(road_raster.layer_metadata, partition_strategy=gps.SpatialPartitionStrategy(32))
masked_ned = tiled_ned.mask(rhode_island_extent.to_polygon, partition_strategy=gps.SpatialPartitionStrategy(32)).convert_data_type(gps.CellType.FLOAT32, 0.0)

### Displaying the NED Data

In [ ]:
ned_tms = masked_ned.tile_to_layout(gps.GlobalLayout(tile_size=256), target_crs=3857)

color_map = gps.ColorMap.build(ned_tms.get_histogram(), 'magma')

layer = gps.TMS.build(ned_tms.pyramid(), color_map)
M.add_layer(TMSRasterData(layer), name="NED-Data")

In [ ]:
M.remove_layer(M.layers[0])

### Caculating Tobler Walking Speeds

In [ ]:
zfactor = gps.geotrellis.zfactor_lat_lng_calculator('Meters')
slope_raster = masked_ned.slope(zfactor)

In [ ]:
tobler_raster = slope_raster.tobler()
adjusted_tobler = (tobler_raster + tiled_nhd + nlcd_pmts) - tiled_path_raster

In [ ]:
friction_with_roads = adjusted_tobler.local_max(road_raster)

In [ ]:
reprojected = friction_with_roads.tile_to_layout(
    target_crs = 3857,
    layout = gps.GlobalLayout(tile_size=256),
    resample_method = gps.ResampleMethod.MAX
).convert_data_type(gps.CellType.FLOAT32, 0.0)

In [ ]:
pyramid = reprojected.pyramid(partition_strategy=gps.SpatialPartitionStrategy(32))

### Displaying the Fricition Layer

In [ ]:
# Building the color map from the histogram of the pyramid
hist_color_map = gps.ColorMap.build(pyramid.get_histogram(), 'magma')
hist_layer = gps.TMS.build(pyramid, hist_color_map)

M.add_layer(TMSRasterData(hist_layer), name="ToblerOSM-from-hist")

In [ ]:
M.remove_layer(M.layers[0])

## Calulating Cost Distance Using the Tobler Layer

### Reading and Formatting Rhode Island Points

In [ ]:
# Downloads the RI points from S3

!curl -o /tmp/ri-points.geojson https://s3.amazonaws.com/geotrellis-test/xterrain/ri-points.geojson

In [ ]:
with fiona.open("ri-points.geojson") as source:
    pop_centers = [shape(f['geometry']) for f in source]

In [ ]:
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

reprojected_pop_centers = [transform(project, geom) for geom in pop_centers]

### Calculating Cost Distance

In [ ]:
cost_distance = gps.cost_distance(3.74 / reprojected,
                                  reprojected_pop_centers,
                                  100000.0)

In [ ]:
cd_pyramid = cost_distance.pyramid()

### Displaying Cost Distance and the Points

In [ ]:
cd_color_map = gps.ColorMap.build(cd_pyramid.get_histogram(), 'viridis')
cd_layer = gps.TMS.build(cd_pyramid, cd_color_map)

M.add_layer(TMSRasterData(cd_layer), name="ToblerOSM-cost-distance")

In [ ]:
M.add_layer(VectorData("ri-points.geojson"),
            name="Manx Population Centers",
            colors=[0xff0000])

In [ ]:
x = 0
while x < len(M.layers):
    M.remove_layer(M.layers[x])
    x += 1